## **Extract from TMDB API** 

Stakeholders need more Data!

- Financial information is missing in the IMDB data loaded previously( e.g. budget or revenue)

This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.
Your stakeholder identified The Movie Database (TMDB) as a great source of financial data (https://www.themoviedb.org/). Thankfully, TMDB offers a free API for programmatic access to their data!

* Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

### Specifications - Financial Data


* Your stakeholder would like you to extract and save the results for movies that meet all of the criteria established in part 1 of the project (You should already have a filtered dataframe saved from part one as a csv.gz file)

* As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

Each year should be saved as a separate .csv.gz file

- Once retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook and explore the following questions.

### Exploratory Data Analysis

* Load in your csv.gz's of results for each year extracted.
    - Concatenate the data into 1 dataframe for the remainder of the analysis.
* Once you have your data from the API, they would like you to perform some light EDA to show:
    - How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
        * Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
    - How many movies are there in each of the certification categories (G/PG/PG-13/R)?
    - What is the average revenue per certification category?
    - What is the average budget per certification category?

NOTE: Load in the cleaned title basics data from the Data folder.
Use the tconst values to extract movie data from The Movie Database's API.
Keep the data sorted in files by startyear to ensure data remains under the git filesize limit.
Combine all years into 1 final CSV and then perform some quick EDA to get overview of data collected (next notebook: EDA)|                                          

In [1]:
# Import the libraries

In [2]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)
import pandas as pd
from tqdm.notebook import tqdm_notebook


In [3]:
# Install tmdbsimple 
!pip install tmdbsimple

In [4]:
import tmdbsimple as tmdb

In [5]:
import json
with open('/Users/purvikansara/.secret/tmdb-apikey.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [6]:

tmdb.API_KEY =  login['api-key']

In [7]:
import os
import pandas as pd
import time
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2006.json',
 'title_basics_cleaned.csv.gz',
 'final_tmdb_data_2004.csv.gz',
 'title_ratings_cleaned.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2002.csv.gz',
 'tmdb_api_results_2002.json',
 'tmdb_api_results_2003.json',
 'final_tmdb_data_2005.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'tmdb_api_results_2004.json',
 'title_akas_cleaned.csv.gz',
 'final_tmdb_data_2003.csv.gz',
 'tmdb_api_results_2005.json']

In [8]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [9]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [10]:
YEARS_TO_GET = list(range(2006,2008))

In [11]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics_cleaned.csv.gz')



In [12]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [13]:
errors = [ ]


In [22]:
YEARS_TO_GET = list(range(2012,2015))

In [23]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):

    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
        
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")




YEARS:   0%|          | 0/3 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/5929 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/7837 [00:00<?, ?it/s]

Movies from 2014:   0%|          | 0/8096 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
errors

In [ ]:
len(errors)